Generalized World Coordinate System (GWCS)
==========================================

This section highlights one of the most powerful capabilities of ASDF; namely
its ability to save arbitrarily complex coordinate transformations with great 
flexibility. This includes the ability to:

- combine transformations using arithmetic operators
- combine transformations in series
- for an arbitrary number of dimensions
- define intermediate coordinates (e.g., the slit plane of a spectrograph)
- parameterize transformations using the parameters as extra dimensions
  (e.g., spectral order, position across a slit, date, etc.)

Contrast this with the FITS WCS system, which works well in imaging and spectra
for standard projections and dispersions, but poorly when dealing with raw data
cointaining complex distortions, or discontinuous transforms (e.g., IFUs), and 
particularly for slitless spectroscopy.

For HST, to achieve sub 0.01 pixel accuracy, 3 different distortion components 
had to be modeled, which were impossible to represent within the FITS WCS 
framework.

We are not able to convey the full capabilities in a few minutes of a tutorial.
This tutorial will illustrate some basics with an imaging example.

We will start with a simple projection and then augment with a distortion model.

The simple projection replicates the basic FITS capabilities using a tangent
projection followed by the appropriate transformation to celestial coordinates.

This involves identifying the point in the detector array that will be the tangent
point, applying the appropriate offset and scaling before applying the tangent
projection, and then transforming the resulting angular coordinates to celestial
coordinates. Schematically:

- Offset detector coordinates to make tangent point in detector have 0, 0 coordinates
- Scale resulting array coordinates to corresponding angular scale.
- Rotate detector coordinates so that north is up
- Apply inverse tangent projection.
- Transform resulting spherical coordinates to corresponding reference point
  in the celestial coordinate system with the appropriate position angle.

These operations are performed using astropy modeling package models.

In [ ]:
import numpy as np
from astropy.modeling import models
from gwcs import wcs
from gwcs import spectroscopy
from gwcs import coordinate_frames as cf
from astropy import coordinates as coord
from astropy import units as u

In [ ]:
# For simplicity we will assume that the detector y-axis is aligned with north, so no 
# rotation of detector coordinates is necessary.
# First step is to defined individual transformation models.
# We assume the detector array is 2000 x 2000 and the tangent point is at (1000, 1000)

# The following constructs a 2D model that shifts both input x and y coordinates by 1000
shift = models.Shift(-1000) & models.Shift(-1000)
# The following constructs a 2D model that scales both input x and y coordinates
# such that the center pixel is 0.1 arcsec in size
scale = models.Scale(0.1 / 3600.) & models.Scale(0.1 / 3600.)
# The following applies an inverse tangent projection
tanproj = models.Pix2Sky_TAN()
# The following moves the spherical coordinates so that the (0, 0) coordinates are moved
# to the supplied RA & Dec coordinates (in degrees), in this case RA = 30, Dec = 45
celest_rot = models.RotateNative2Celestial(30., 45., 180.) # last arg is always 180.

# The following is the net transformation from pixel coordinates to celestial coordinates
transform = shift | scale | tanproj | celest_rot

In [ ]:
# Now we define the frames of reference for the WCS
detector_frame = cf.Frame2D(name='detector', axes_names=('x','y'), unit=(u.pix, u.pix))
sky_frame = cf.CelestialFrame(reference_frame=coord.ICRS(), unit=(u.deg, u.deg))
wcsobj = wcs.WCS([(detector_frame, transform), (sky_frame, None)])

In [ ]:
wcsobj(1000, 1000)

In [ ]:
wcsobj(1000, 1001)

In [ ]:
1/3600,

In [ ]:
wcsobj(1000, 1000 + 36000 * 5)

Slitless Spectrograph Example
-----------------------------

We will consider building a WCS for a slitless spectrograph. For simplicity's sake,
This will be done as a 1D spectrograph, although the same concepts can be generalized
to 2D. This is an interesting case since given a pixel in the detector mapping it to
a wavelength is not possible unless the position of the source is known. In other 
words, one must know either position or wavelength, to determine the other. Generally
for the slitless the position is determined by other means in order for the wavelenth
to be determined.

This WCS will be based on a simple spectrograph design, taken from this 
[site](http://www.astrosurf.com/buil/us/stage/calcul/design_us.htm), since
the relevant design parameters make it simple to generate the WCS.

The relevant parameters are:

telescope focal length: 1000 mm
collimator focal length: 100 mm
camera focal length: 50 mm
grating density: 600 grooves/mm
nominal grating incident angle: 28 degrees

The optical axis of the collimator is presumed to correspond to 0 relative
angular coordinate from the telescope. Given a relative spatial offset in arcseconds
the computed incident grating angle becomes:

`incident_grating_angle = (relative_spatial_angle * telescope_focal_length / collimator_focal_length) + design_incident_grating_angle`

The grating equation is used to compute the outgoing angle from the grating angle
given the incoming angle and wavelength. The GWCS package contains a model that does this
in 3D. It is easily used for the 1D case by setting the beta parameter to 0. Specifically

`gwcs.spectroscopy.AnglesFromGratingEquation3D(groove_density=grating_density * 1000, spectral_order=1)`

Since this class expects groove_density in grooves per meter.

It expects `wavelength`, `alpha_in`, `beta_in` as arguments and returns 
alpha_out, beta_out, and gamma_out. We only care about alpha out in this case,
which corresponds to beta in the design on the referenced page.

We presume the design beta hits the detector at its reference pixel. 
We presume a 2000 pixel linear detector array so that the reference pixel is at 1000.
So the pixel that the outgoing ray hits is computed by the following:

`pixel_coord = (alpha_out - beta) * camera_focal_length / plate_scale + 1000`

Now for the actual code, with appropriate handling of the units. We will define 
different intermediate transforms (great for debugging!)

In [ ]:
deg2rad = np.pi / 180.
TELESCOPE_FOCAL_LENGTH = 1200 # in mm
COLLIMATOR_FOCAL_LENGTH = 100 # in mm
CAMERA_FOCAL_LENGTH = 50 # in mm
GRATING_DENSITY = 600000 # grooves / meter
DESIGN_INCIDENT_GRATING_ANGLE = 28 * deg2rad # in radians
# DESIGN_REFERENCE_GRATING_EXIT_ANGLE = -7.94 * deg2rad # in radians; web site appears
# to have the wrong value
# DESIGN_REFERENCE_GRATING_EXIT_ANGLE = -7.99113190973 * deg2rad # in radians
# DESIGN_REFERENCE_GRATING_EXIT_ANGLE = -9.092392760682835 * deg2rad
DESIGN_REFERENCE_GRATING_EXIT_ANGLE = -8.017269026393847 * deg2rad
REFERENCE_PIXEL = 1000
REFERENCE_WAVELENGTH = 5500 # in Angstroms
PLATE_SCALE = 100 # pixels / mm



# The GWCS transform ultimately has two input coordinates: wavelength and relative sky position.
# So the first transform has these two coordinates as inputs, but only the sky position is transformed 
# before the grating. Hence the use of the Identity transform for the wavelength coordinate.
# The grating class from GWCS assumes the input angle is actually the sine of the input angle
# The transform converts Angstrom, degree inputs to meters, radians
pre_grating_transform =  (models.Scale(1e-10) & # Convert Angstroms to meters
                          (models.Scale(TELESCOPE_FOCAL_LENGTH / COLLIMATOR_FOCAL_LENGTH * deg2rad) | 
                          models.Shift(DESIGN_INCIDENT_GRATING_ANGLE))) #| 
#                          models.math.SinUfunc()))
                        
# Use the grating equation to define a model that computs the exiting angle given wavelength and
# incident angle.
grating_transform = (((models.Mapping((0,), 2) | models.Scale(GRATING_DENSITY)) 
                      - (models.Mapping((1,), 2) | models.math.SinUfunc())) |
                      models.math.ArcsinUfunc())

# Transform exit angle into pixel coordinates
post_grating_transform = (models.Shift(-DESIGN_REFERENCE_GRATING_EXIT_ANGLE) | 
                          models.Scale(CAMERA_FOCAL_LENGTH * PLATE_SCALE) |
                          models.Shift(1000))

In [ ]:
nettrans = pre_grating_transform | grating_transform | post_grating_transform

In [ ]:
skywav2pix = wcs.WCS([("skywav", nettrans),
                      ("detector", None)])

In [ ]:
skywav2pix(REFERENCE_WAVELENGTH, 0)

In [ ]:
# Now fix to a position
source1_wave2pix = skywav2pix.fix_inputs({1: 0.1})
pix = source1_wave2pix(5500)
pix

In [ ]:
# Derive A related transformation that give wavelength given pixel location
# and source position, i.e., effectively an inverse transform.

post_grating_transform_inverse = (models.Shift(-1000) | 
                                  models.Scale(1 / (CAMERA_FOCAL_LENGTH * PLATE_SCALE)) |
                                  models.Shift(DESIGN_REFERENCE_GRATING_EXIT_ANGLE))
grating_transform_to_wavelength = (((models.Mapping((0,), 2) | models.math.SinUfunc())
                  + (models.Mapping((1,), 2) | models.math.SinUfunc())) |
                                     models.Scale(1. / GRATING_DENSITY * 1e10)) # Convert wavelengt in meters to Angstroms
pre_grating_simple_transform = (models.Scale(TELESCOPE_FOCAL_LENGTH / COLLIMATOR_FOCAL_LENGTH * deg2rad) | 
                                models.Shift(DESIGN_INCIDENT_GRATING_ANGLE))

In [ ]:
# Construct transforms and GWCS for a known source position
source1_transform = models.fix_inputs(nettrans, {1: 0})

In [ ]:
# Normally we would assign the net inverse transform to a variable, but some odd notebook
# interaction prevents this, so the net transform is provided as an expression instead.
source1_inverse = models.fix_inputs((post_grating_transform_inverse & pre_grating_simple_transform | 
                      grating_transform_to_wavelength), {1: 0})

In [ ]:
source1_inverse(1000)

In [ ]:
# Construct two different general GWCS models, one for each direction
source2pix = wcs.WCS([('source_frame', nettrans),
                        ('detector_frame', None)])
pix2source = wcs.WCS([('detector_frame', (post_grating_transform_inverse & pre_grating_simple_transform | 
                      grating_transform_to_wavelength)),
                   ('source_frame', None)])

In [ ]:
source2pix(5500, 0)

In [ ]:
pix2source(1000, 0)

In [ ]:
source1_wave2pix = source2pix.fix_inputs({1: 0})
source1_pix2wave = pix2source.fix_inputs({1: 0})

In [ ]:
source1_wave2pix(5500)

In [ ]:
source1_pix2wave(1000)

Important Points
----------------

- All the defined transforms and GWCS objects (general and source-specific) are easily
  serialized to ASDF.
- This illustrates that one can define general transforms for a slitless 1d spectrograph
  for all possible source positions can be stored in the data file. 
- When the source positions are identified (perhapslater by more analysis or merging
  imaging data) specific dispersion transformations can be obtained from the general
  transforms easily without repeating a possibly complex general transform (e.g.,
  optical distortions can be folded into the tranforms), for each source identified
- Instead of using wavelength tables, one for each source identified, a common transform
  is available, and may be tweaked in one place (if not the source position) and
  automatically applied to all source dispersion transforms.
- The wavelength table approach is generated elsewhere; changes require rerunning
  the software that generated it whereas it is easy to generate a wavelength table
  from the transforms.
- The 1d example can be generalized to 2D slitless in the same style.
- The general slitless approach is also suitable for multiple object spectrographs
  where slits may be located at arbitrary locations in the focal plane, such as
  those that use shutters such as the JWST NIRSPEC MOS mode.
- The ability to define transforms that fold in parameters as extra coordinates
  that may vary between datasets such as order, temperature, date, etc. allows
  use of a single GWCS model that can be applied to all such variations without
  the need to store different GWCS models for each data set.